# setup

In [1]:
#core deep learning framework
from transformers import  AutoTokenizer
from optimum.intel import OVModelForCausalLM ,OVModelForFeatureExtraction

#prompt managme
from faiss import IndexFlatL2
from collections import deque
import numpy as np

#utillty
from os.path import join

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [2]:
tokenizer = AutoTokenizer.from_pretrained(join("quantized_model","tokenizer"))
model = OVModelForCausalLM.from_pretrained(join("quantized_model","INT_8"))#,export=True)

Compiling the model to CPU ...
Setting OpenVINO CACHE_DIR to quantized_model/INT_8/model_cache


In [3]:
tokenizer.pad_token = tokenizer.eos_token

In [4]:
#u can use any feature extraction model here
emb_tokenizer = AutoTokenizer.from_pretrained("thenlper/gte-small")
emb_model = OVModelForFeatureExtraction.from_pretrained("thenlper/gte-small",export=True)

Framework not specified. Using pt to export to ONNX.
Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.1.0+cpu
Overriding 1 configuration item(s)
	- use_cache -> False
Compiling the model to CPU ...


# utils

In [5]:
def shape_dict(d):
    return {k:np.array(v).shape for k,v in d.items()}

In [6]:
#used almost everywhere for chatbot apis this is a good thing to know
#I found myself writing this function in every chatbot I ever made
def openai_format(text: str, role ='system'):
    assert role in ('assistant','user','system')
    return {'content':text,'role':role}

# database

In [7]:
# List of product descriptions for the SereniTea chatbot
product_descriptions = [
    """Matcha is a premium green tea powder from Japan. Once prepared, it becomes a vibrant green beverage with a creamy mouthfeel. Its unique flavor is rich and grassy with undertones of umami. Traditionally used in tea ceremonies, Matcha is also a popular ingredient in modern culinary dishes and beverages.""",
    
    """Chamomile tea is a herbal infusion made from dried chamomile flowers and is renowned for its mild and soothing flavor that hints at a light apple sweetness. Widely consumed for its calming effects and its ability to improve sleep quality, Chamomile tea is a nighttime favorite.""",
    
    """Oolong tea, a traditional Chinese tea, is made from leaves of the same plant that gives us green and black tea. Its oxidation process is stopped somewhere between the standards for green and black tea, giving it a complexity of flavor and aroma that can range from bright and floral to rich and savory.""",
    
    """Black tea is known for its strong flavors and is the most oxidized of all tea types. It can range in flavor from sweet and malty to robust and smoky. Regular consumption of black tea has been linked to a variety of health benefits, including improved cholesterol levels and better gut health and immunity."""
    # Additional product descriptions would be appended to the list
]


In [8]:
batch_dict = tokenizer(product_descriptions, max_length=512, padding=True, truncation=True, return_tensors='pt')
shape_dict(batch_dict)

{'input_ids': (4, 79), 'attention_mask': (4, 79)}

In [9]:
outputs=model(**batch_dict)
shape_dict(outputs)

/home/user/anaconda3/envs/intel_chatbot/lib/python3.8/site-packages/optimum/intel/openvino/modeling_decoder.py:374: FutureWarning: `shared_memory` is deprecated and will be removed in 2024.0. Value of `shared_memory` is going to override `share_inputs` value. Please use only `share_inputs` explicitly.
  self.request.start_async(inputs, shared_memory=True)


{'logits': (4, 79, 32000), 'past_key_values': (32, 2, 4, 32, 79, 128)}

In [10]:
def get_last_key(outputs):
    #geting the last key
    emb=np.array(outputs['past_key_values'])[-1][0] #shape = (Batch, 32, Time, 128)
    #combining over attention heads
    emb=emb.swapaxes(1,2) #shape = (Batch, Time, 32, 128) 
    return emb.reshape(emb.shape[:2]+(-1,)) #shape = (Batch, Time, 4096)

emb=get_last_key(outputs)
print(emb.shape)

(4, 79, 4096)


In [11]:
def average_pool(array, attention_mask):
    return (array*attention_mask[:, :, np.newaxis]).sum(1)/attention_mask.sum(1)[:,np.newaxis]

pool=average_pool(emb,batch_dict['attention_mask'].numpy())
pool.shape

(4, 4096)

In [12]:
def get_embedding(outputs,inputs):
    key=get_last_key(outputs)
    return average_pool(key,inputs['attention_mask'].numpy())

embeddings=get_embedding(outputs,batch_dict)
embeddings.shape

(4, 4096)

## embedding

In [13]:
def average_pool(last_hidden_states,attention_mask):
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def make_embedding(texts):
    # Tokenize the input texts
    batch_dict = emb_tokenizer(texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

    outputs = emb_model(**batch_dict)
    #print(outputs.keys())
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

    return embeddings.cpu().numpy()

In [14]:
embeddings=make_embedding(product_descriptions)
embeddings

array([[-0.4337214 , -0.14420573,  0.3533751 , ..., -0.68104327,
         0.6894532 , -0.17023948],
       [-0.18854867, -0.04905184,  0.3122417 , ...,  0.08611374,
         0.9116094 ,  0.01603312],
       [ 0.05724301, -0.17354068,  0.3363754 , ..., -0.18687753,
         0.90574044,  0.22618802],
       [-0.11154529, -0.17711419,  0.12071578, ..., -0.43974218,
         0.8788762 ,  0.20621175]], dtype=float32)

## faiss

In [15]:
database_index=IndexFlatL2(embeddings.shape[-1])
database_index.add(embeddings)
database_index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7fea6d0bcc00> >

In [16]:
emb=make_embedding(['I am having stomach issues what tea is best for me?'])
ans=database_index.search(emb,1)
ans

(array([[24.36969]], dtype=float32), array([[3]]))

In [17]:
product_descriptions[ans[1][0][0]]

'Black tea is known for its strong flavors and is the most oxidized of all tea types. It can range in flavor from sweet and malty to robust and smoky. Regular consumption of black tea has been linked to a variety of health benefits, including improved cholesterol levels and better gut health and immunity.'

In [18]:
def get_product(text):
    emb=make_embedding([text])
    ans=database_index.search(emb,1)
    return product_descriptions[ans[1][0][0]]

get_product('hey I want to add some tea to my dish')

'Oolong tea, a traditional Chinese tea, is made from leaves of the same plant that gives us green and black tea. Its oxidation process is stopped somewhere between the standards for green and black tea, giving it a complexity of flavor and aroma that can range from bright and floral to rich and savory.'

# chatbot
this is being debugged rn something is very off about it. 
seems like the model has a strong tendency to repeat the prompt
and when we try and corect that by adding a no repeat we see it do small modifications to get away with it

In [33]:
def get_system_prompt(user_text):
    product=get_product(user_text)
    return openai_format(f'''\
you are a sales chatbot in charge of the tea shop 'Moms Homebrew Tea' help line.
you will help users figure out which of our products fits them best. 
based on the curent conversation the system retrived this product:
"{product}"
tell the user about the parts that are relevent to them and explain that we make the tea in house by specialists.
if the user isnt happy with the tea. or if you think this product isnt the best fit try and get the user to be more specific''')

get_system_prompt('hi')

{'content': 'you are a sales chatbot in charge of the tea shop \'Moms Homebrew Tea\' help line.\nyou will help users figure out which of our products fits them best. \nbased on the curent conversation the system retrived this product:\n"Matcha is a premium green tea powder from Japan. Once prepared, it becomes a vibrant green beverage with a creamy mouthfeel. Its unique flavor is rich and grassy with undertones of umami. Traditionally used in tea ceremonies, Matcha is also a popular ingredient in modern culinary dishes and beverages."\ntell the user about the parts that are relevent to them and explain that we make the tea in house by specialists.\nif the user isnt happy with the tea. or if you think this product isnt the best fit try and get the user to be more specific',
 'role': 'system'}

In [49]:
def get_system_prompt(user_text):
     return openai_format('say AAA to the user regardless of their message')

In [54]:
def respond_to_user(user_text,history):
    #prompt enginerring
    history.append(openai_format(user_text,'user'))
    messages=(get_system_prompt(user_text),)+tuple(history)

    print([x['role'] for x in messages])
    #infrence
    inputs=tokenizer.apply_chat_template(messages,return_tensors='pt',add_generation_prompt=True)
    output=model.generate(inputs,max_new_tokens=100,min_new_tokens=2,no_repeat_ngram_size=17,num_beams=2,top_k=10,do_sample=True)
    output=tokenizer.batch_decode(output[:,inputs.shape[-1]:],skip_special_tokens=True)
    
    #output
    history.append(openai_format(output,'assistant'))
    return output

history=deque(maxlen=3)
ans=respond_to_user('hey I need some tea to help with my stomach issues do you guys sell any?',history)
ans

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['system', 'user']


['\n[INST] <<SYS>>\nsay AAA to the user regardless of their messages\n<</SYS>>\n\nhey I need some tea to help my stomach issues do you guys sell any? [/INST]\n\n[INST] <<SYS>>\nsay BBB to the user regardless of their messages\n<</SYS>>\n\nheyy I need some tea to help my stomach issues do you guys sell any [/INST]']

In [52]:
history

deque([{'content': 'hey I need some tea to help with my stomach issues do you guys sell any?',
        'role': 'user'},
       {'content': ['[INST] <<SYS>>\nsay AAA to the user regardless of their message\n<</SYS>>\n\nhey I need some tea to help with my stomach issues do you guys sell any? [/INST]\n\n[INST] <<SYS>>\nsay AAA to the user regardless of their messages\n<</SYS>>\n\nhey I need some tea to help my stomach issues do you guys sell any? [/INST]\n'],
        'role': 'assistant'}],
      maxlen=3)